In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
url_geo='/content/drive/MyDrive/Python_text/Data_Code/section6/londongeo_clean.csv'
url_bor='/content/drive/MyDrive/Python_text/Data_Code/section6/london_boroughs.csv'

In [4]:
df_geo = pd.read_csv(url_geo)

In [6]:
df_geo.head()

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.079100
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W3,"Ealing, Hammersmith & West London College, St ...","(51.49153695, -0.21497089652160112, 0.0)",51.491537,-0.214971
2,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W4,"Ealing, Hammersmith & West London College, St ...","(51.49153695, -0.21497089652160112, 0.0)",51.491537,-0.214971
3,Aldgate,City,LONDON,TQ334813,EC3,"City of London, Greater London, England, Unite...","(51.5156177, -0.0919983, 0.0)",51.515618,-0.091998
4,Aldwych,Westminster,LONDON,TQ307810,WC2,"Westminster, Millbank, City of Westminster, Lo...","(51.5004439, -0.1265398, 0.0)",51.500444,-0.126540


In [7]:
df_bor = pd.read_csv(url_bor)

In [9]:
df_bor.head()

,New code,Area name,InnerOuter,PopulationEstimate_2014,HouseholdEstimate_2014,InlandArea_Hectares,PopulationDensity2013,AverageAge2013,ProportionPopulation0to15,ProportionWorkingage2013,Proportion65over2013,ResidentPopulation_abroad2013,LargestMigrantPopulation2011,PercentLargestMigrant2011,OverseasNationalsEntering2013,Employmentrate2013,MaleEmployment2013,FemaleEmployment2013,WorkingAgeBenefits,WorkingAgeDisability,ProportionDegreeHolders2013,GrossAnnualPay2014,GrossAnnualPay_Male2014,GrossAnnualPay_Female2014,ActiveBusinesses2013,MedianHousePrice2013,NewHomes,Greenspace,CarbonEmissions,NumberCars,CarsPerHouseold,AveragePublicTransportAccessibility 2012,MultipleDeprivationScore,MaleLifeExpectancy,FemaleLifeExpectancy,TeenageConception,LifeSatisfaction,Worthwhileness,Happiness,PoliticalControl
0,E09000001,City of London,Inner London,"8,000","4,632",290.4,27.5,41.3,7.9,77.5,14.5,37.4,United States,2.8,712,81.2,77.0,70.0,13.7,14.8,65.3,"_54,591","_62,263","_42,904","17,775","5,95,000",440,4.8,"1,499.00","1,692",0.4,7.6,262,NaN,NaN,NaN,8.20,7.89,7.24,Lab
1,E09000002,Barking and Dagenham,Outer London,"2,01,000","75,327",3610.8,55.7,33.2,26.1,63.8,10.1,35.8,Nigeria,4.7,"4,405",63.2,73.2,53.5,7.6,22.6,21.2,"_32,413","_36,122","_29,147","5,055","1,80,000",730,33.6,780.8,"56,966",0.8,3.0,22,77.7,82.4,35.4,6.93,7.33,7.04,Lab
2,E09000003,Barnet,Outer London,"3,84,600","1,46,819",8674.8,44.3,36.9,20.9,65.5,13.6,35.9,India,3.1,"9,406",72.5,80.0,65.0,8.2,11.8,43.8,"_31,010","_36,479","_25,640","23,135","3,46,000","1,110",41.3,"1,542.00","1,44,717",1.1,3.0,176,81.9,85.0,14.7,7.39,7.69,7.53,Cons
3,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons
4,E09000005,Brent,Outer London,"3,22,800","1,14,465",4323.3,74.7,35.3,20.5,68.7,10.8,53.3,India,9.2,"16,696",65.9,75.3,56.0,7.2,18.7,36.9,"_29,489","_30,862",.,"13,915","3,50,000",730,21.9,"1,358.20","87,802",0.8,3.7,35,80.0,84.9,19.6,7.31,7.51,7.22,Lab


In [11]:
df2 = df_bor.rename({'New code': 'Code', 'Area name': 'Area'}, axis=1)  #original: new name


In [12]:
df2.head()

,Code,Area,InnerOuter,PopulationEstimate_2014,HouseholdEstimate_2014,InlandArea_Hectares,PopulationDensity2013,AverageAge2013,ProportionPopulation0to15,ProportionWorkingage2013,Proportion65over2013,ResidentPopulation_abroad2013,LargestMigrantPopulation2011,PercentLargestMigrant2011,OverseasNationalsEntering2013,Employmentrate2013,MaleEmployment2013,FemaleEmployment2013,WorkingAgeBenefits,WorkingAgeDisability,ProportionDegreeHolders2013,GrossAnnualPay2014,GrossAnnualPay_Male2014,GrossAnnualPay_Female2014,ActiveBusinesses2013,MedianHousePrice2013,NewHomes,Greenspace,CarbonEmissions,NumberCars,CarsPerHouseold,AveragePublicTransportAccessibility 2012,MultipleDeprivationScore,MaleLifeExpectancy,FemaleLifeExpectancy,TeenageConception,LifeSatisfaction,Worthwhileness,Happiness,PoliticalControl
0,E09000001,City of London,Inner London,"8,000","4,632",290.4,27.5,41.3,7.9,77.5,14.5,37.4,United States,2.8,712,81.2,77.0,70.0,13.7,14.8,65.3,"_54,591","_62,263","_42,904","17,775","5,95,000",440,4.8,"1,499.00","1,692",0.4,7.6,262,NaN,NaN,NaN,8.20,7.89,7.24,Lab
1,E09000002,Barking and Dagenham,Outer London,"2,01,000","75,327",3610.8,55.7,33.2,26.1,63.8,10.1,35.8,Nigeria,4.7,"4,405",63.2,73.2,53.5,7.6,22.6,21.2,"_32,413","_36,122","_29,147","5,055","1,80,000",730,33.6,780.8,"56,966",0.8,3.0,22,77.7,82.4,35.4,6.93,7.33,7.04,Lab
2,E09000003,Barnet,Outer London,"3,84,600","1,46,819",8674.8,44.3,36.9,20.9,65.5,13.6,35.9,India,3.1,"9,406",72.5,80.0,65.0,8.2,11.8,43.8,"_31,010","_36,479","_25,640","23,135","3,46,000","1,110",41.3,"1,542.00","1,44,717",1.1,3.0,176,81.9,85.0,14.7,7.39,7.69,7.53,Cons
3,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons
4,E09000005,Brent,Outer London,"3,22,800","1,14,465",4323.3,74.7,35.3,20.5,68.7,10.8,53.3,India,9.2,"16,696",65.9,75.3,56.0,7.2,18.7,36.9,"_29,489","_30,862",.,"13,915","3,50,000",730,21.9,"1,358.20","87,802",0.8,3.7,35,80.0,84.9,19.6,7.31,7.51,7.22,Lab


In [ ]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 4.9MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149805 sha256=af754e95e3076abf2a40775104e69fdaef96ec4f9d4c649a57605256595ce434
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-levenshtein


In [ ]:
from fuzzywuzzy import process, fuzz

Create two new columns common for both dataframes--> 'dummy'

In [ ]:
#df_geo['dummy'] = True 
#df2['dummy'] = True
 


In [13]:
df_geo['dummy']=True
df2['dummy']=True

In [14]:
df2.head()

,Code,Area,InnerOuter,PopulationEstimate_2014,HouseholdEstimate_2014,InlandArea_Hectares,PopulationDensity2013,AverageAge2013,ProportionPopulation0to15,ProportionWorkingage2013,Proportion65over2013,ResidentPopulation_abroad2013,LargestMigrantPopulation2011,PercentLargestMigrant2011,OverseasNationalsEntering2013,Employmentrate2013,MaleEmployment2013,FemaleEmployment2013,WorkingAgeBenefits,WorkingAgeDisability,ProportionDegreeHolders2013,GrossAnnualPay2014,GrossAnnualPay_Male2014,GrossAnnualPay_Female2014,ActiveBusinesses2013,MedianHousePrice2013,NewHomes,Greenspace,CarbonEmissions,NumberCars,CarsPerHouseold,AveragePublicTransportAccessibility 2012,MultipleDeprivationScore,MaleLifeExpectancy,FemaleLifeExpectancy,TeenageConception,LifeSatisfaction,Worthwhileness,Happiness,PoliticalControl,dummy
0,E09000001,City of London,Inner London,"8,000","4,632",290.4,27.5,41.3,7.9,77.5,14.5,37.4,United States,2.8,712,81.2,77.0,70.0,13.7,14.8,65.3,"_54,591","_62,263","_42,904","17,775","5,95,000",440,4.8,"1,499.00","1,692",0.4,7.6,262,NaN,NaN,NaN,8.20,7.89,7.24,Lab,True
1,E09000002,Barking and Dagenham,Outer London,"2,01,000","75,327",3610.8,55.7,33.2,26.1,63.8,10.1,35.8,Nigeria,4.7,"4,405",63.2,73.2,53.5,7.6,22.6,21.2,"_32,413","_36,122","_29,147","5,055","1,80,000",730,33.6,780.8,"56,966",0.8,3.0,22,77.7,82.4,35.4,6.93,7.33,7.04,Lab,True
2,E09000003,Barnet,Outer London,"3,84,600","1,46,819",8674.8,44.3,36.9,20.9,65.5,13.6,35.9,India,3.1,"9,406",72.5,80.0,65.0,8.2,11.8,43.8,"_31,010","_36,479","_25,640","23,135","3,46,000","1,110",41.3,"1,542.00","1,44,717",1.1,3.0,176,81.9,85.0,14.7,7.39,7.69,7.53,Cons,True
3,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
4,E09000005,Brent,Outer London,"3,22,800","1,14,465",4323.3,74.7,35.3,20.5,68.7,10.8,53.3,India,9.2,"16,696",65.9,75.3,56.0,7.2,18.7,36.9,"_29,489","_30,862",.,"13,915","3,50,000",730,21.9,"1,358.20","87,802",0.8,3.7,35,80.0,84.9,19.6,7.31,7.51,7.22,Lab,True


Combine both df_geo and df2 on the basis of the common column
Drop dummy

In [ ]:
#df = pd.merge(df_geo,df2, on='dummy')
 


In [15]:
df=pd.merge(df_geo,df2,on='dummy')

In [16]:
df.drop('dummy',axis=1,inplace=True)

In [ ]:
#df.drop('dummy', axis=1, inplace=True)
#df

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude,matches,Code,Area,InnerOuter,PopulationEstimate_2014,HouseholdEstimate_2014,InlandArea_Hectares,PopulationDensity2013,AverageAge2013,ProportionPopulation0to15,ProportionWorkingage2013,Proportion65over2013,ResidentPopulation_abroad2013,LargestMigrantPopulation2011,PercentLargestMigrant2011,OverseasNationalsEntering2013,Employmentrate2013,MaleEmployment2013,FemaleEmployment2013,WorkingAgeBenefits,WorkingAgeDisability,ProportionDegreeHolders2013,GrossAnnualPay2014,GrossAnnualPay_Male2014,GrossAnnualPay_Female2014,ActiveBusinesses2013,MedianHousePrice2013,NewHomes,Greenspace,CarbonEmissions,NumberCars,CarsPerHouseold,AveragePublicTransportAccessibility 2012,MultipleDeprivationScore,MaleLifeExpectancy,FemaleLifeExpectancy,TeenageConception,LifeSatisfaction,Worthwhileness,Happiness,PoliticalControl
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000001,City of London,Inner London,"8,000","4,632",290.4,27.5,41.3,7.9,77.5,14.5,37.4,United States,2.8,712,81.2,77.0,70.0,13.7,14.8,65.3,"_54,591","_62,263","_42,904","17,775","5,95,000",440,4.8,"1,499.00","1,692",0.4,7.6,262,NaN,NaN,NaN,8.20,7.89,7.24,Lab
1,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000002,Barking and Dagenham,Outer London,"2,01,000","75,327",3610.8,55.7,33.2,26.1,63.8,10.1,35.8,Nigeria,4.7,"4,405",63.2,73.2,53.5,7.6,22.6,21.2,"_32,413","_36,122","_29,147","5,055","1,80,000",730,33.6,780.8,"56,966",0.8,3.0,22,77.7,82.4,35.4,6.93,7.33,7.04,Lab
2,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000003,Barnet,Outer London,"3,84,600","1,46,819",8674.8,44.3,36.9,20.9,65.5,13.6,35.9,India,3.1,"9,406",72.5,80.0,65.0,8.2,11.8,43.8,"_31,010","_36,479","_25,640","23,135","3,46,000","1,110",41.3,"1,542.00","1,44,717",1.1,3.0,176,81.9,85.0,14.7,7.39,7.69,7.53,Cons
3,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons
4,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000005,Brent,Outer London,"3,22,800","1,14,465",4323.3,74.7,35.3,20.5,68.7,10.8,53.3,India,9.2,"16,696",65.9,75.3,56.0,7.2,18.7,36.9,"_29,489","_30,862",.,"13,915","3,50,000",730,21.9,"1,358.20","87,802",0.8,3.7,35,80.0,84.9,19.6,7.31,7.51,7.22,Lab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12502,Wormwood Scrubs,Hammersmith and Fulham,LONDON,TQ225815,W12,"Hammersmith, London, Greater London, England, ...","(51.4920377, -0.2236401, 0.0)",51.492038,-0.22364,Hammersmith and Fulham,E09000029,Sutton,Outer London,"1,95,600","79,536",4384.7,44.6,38.3,20.2,65.0,14.9,21.2,Sri Lanka,1.8,"1,409",79.8,85.8,74.0,7.2,17.7,33.5,"_25,791","_26,989","_24,685","7,805","2,50,000",300,32.0,819.7,"91,266",1.2,2.7,196,80.8,84.1,25.8,7.43,7.60,7.25,Lib Dem
12503,Wormwood Scrubs,Hammersmith and Fulham,LONDON,TQ225815,W12,"Hammersmith, London, Greater London, England, ...","(51.4920377, -0.2236401, 0.0)",51.492038,-0.22364,Hammersmith and Fulham,E09000030,Tower Hamlets,Inner London,"2,77,900","1,10,630",1978.1,140.5,31.1,19.6,74.5,5.9,42.5

Measure similarity between Borough and Area

In [ ]:
df['Ratio'] = df[['Borough','Area']].apply(lambda x:fuzz.ratio(x.Borough, x.Area), axis=1)
df['Partial_Ratio'] = df[['Borough','Area']].apply(lambda x:fuzz.partial_ratio(x.Borough, x.Area), axis=1)
df['Token_Sort_Ratio'] = df[['Borough','Area']].apply(lambda x:fuzz.token_sort_ratio(x.Borough, x.Area), axis=1)
df['Token_Set_Ratio'] = df[['Borough','Area']].apply(lambda x:fuzz.token_set_ratio(x.Borough, x.Area), axis=1)
 
df

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude,matches,Code,Area,InnerOuter,PopulationEstimate_2014,HouseholdEstimate_2014,InlandArea_Hectares,PopulationDensity2013,AverageAge2013,ProportionPopulation0to15,ProportionWorkingage2013,Proportion65over2013,ResidentPopulation_abroad2013,LargestMigrantPopulation2011,PercentLargestMigrant2011,OverseasNationalsEntering2013,Employmentrate2013,MaleEmployment2013,FemaleEmployment2013,WorkingAgeBenefits,WorkingAgeDisability,ProportionDegreeHolders2013,GrossAnnualPay2014,GrossAnnualPay_Male2014,GrossAnnualPay_Female2014,ActiveBusinesses2013,MedianHousePrice2013,NewHomes,Greenspace,CarbonEmissions,NumberCars,CarsPerHouseold,AveragePublicTransportAccessibility 2012,MultipleDeprivationScore,MaleLifeExpectancy,FemaleLifeExpectancy,TeenageConception,LifeSatisfaction,Worthwhileness,Happiness,PoliticalControl,Ratio,Partial_Ratio,Token_Sort_Ratio,Token_Set_Ratio
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000001,City of London,Inner London,"8,000","4,632",290.4,27.5,41.3,7.9,77.5,14.5,37.4,United States,2.8,712,81.2,77.0,70.0,13.7,14.8,65.3,"_54,591","_62,263","_42,904","17,775","5,95,000",440,4.8,"1,499.00","1,692",0.4,7.6,262,NaN,NaN,NaN,8.20,7.89,7.24,Lab,19,21,20,20
1,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000002,Barking and Dagenham,Outer London,"2,01,000","75,327",3610.8,55.7,33.2,26.1,63.8,10.1,35.8,Nigeria,4.7,"4,405",63.2,73.2,53.5,7.6,22.6,21.2,"_32,413","_36,122","_29,147","5,055","1,80,000",730,33.6,780.8,"56,966",0.8,3.0,22,77.7,82.4,35.4,6.93,7.33,7.04,Lab,26,28,33,33
2,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000003,Barnet,Outer London,"3,84,600","1,46,819",8674.8,44.3,36.9,20.9,65.5,13.6,35.9,India,3.1,"9,406",72.5,80.0,65.0,8.2,11.8,43.8,"_31,010","_36,479","_25,640","23,135","3,46,000","1,110",41.3,"1,542.00","1,44,717",1.1,3.0,176,81.9,85.0,14.7,7.39,7.69,7.53,Cons,25,33,27,27
3,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,50,100,55,100
4,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.07910,"Bexley, Greenwich",E09000005,Brent,Outer London,"3,22,800","1,14,465",4323.3,74.7,35.3,20.5,68.7,10.8,53.3,India,9.2,"16,696",65.9,75.3,56.0,7.2,18.7,36.9,"_29,489","_30,862",.,"13,915","3,50,000",730,21.9,"1,358.20","87,802",0.8,3.7,35,80.0,84.9,19.6,7.31,7.51,7.22,Lab,35,60,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12502,Wormwood Scrubs,Hammersmith and Fulham,LONDON,TQ225815,W12,"Hammersmith, London, Greater London, England, ...","(51.4920377, -0.2236401, 0.0)",51.492038,-0.22364,Hammersmith and Fulham,E09000029,Sutton,Outer London,"1,95,600","79,536",4384.7,44.6,38.3,20.2,65.0,14.9,21.2,Sri Lanka,1.8,"1,409",79.8,85.8,74.0,7.2,17.7,33.5,"_25,791","_26,989","_24,685","7,805","2,50,000",300,32.0,819.7,"91,266",1.2,2.7,196,80.8,84.1,25.8,7.43,7.60,7.25,Lib Dem,14,33,14,14
12503,Wormwood Scrubs,Hammersmith and Fulham,LONDON,TQ225815,W12,"Hammersmith, London, Greater London, England, ...","(51.4920377, -0.2236401, 

See similarity between Borough and Area in ascending order

In [ ]:
df['Rank_Token_Set_Ratio'] = df.groupby('Borough')['Token_Set_Ratio'].rank(ascending=False, method='dense')
df[['Borough','Area', 'Token_Set_Ratio']].loc[df.Rank_Token_Set_Ratio==1]

,Borough,Area,Token_Set_Ratio
3,"Bexley, Greenwich",Bexley,100
10,"Bexley, Greenwich",Greenwich,100
41,"Ealing, Hammersmith and Fulham",Ealing,100
45,"Ealing, Hammersmith and Fulham",Hammersmith and Fulham,100
74,"Ealing, Hammersmith and Fulham",Ealing,100
...,...,...,...
12400,"Redbridge, Waltham Forest",Redbridge,100
12405,"Redbridge, Waltham Forest",Waltham Forest,100
12410,Barnet,Barnet,100
12451,Greenwich,Greenwich,100


Assign 'Area' of df2 to a new column 'Most_Similar' of df_geo

In [ ]:
df_geo['Most_Similar'] = df_geo.Borough.apply(lambda x : [process.extract(x, df2.Area, limit=1)][0][0][0])
 
df_geo[['Borough', 'Most_Similar']]

,Borough,Most_Similar
0,"Bexley, Greenwich",Bexley
1,"Ealing, Hammersmith and Fulham",Hammersmith and Fulham
2,"Ealing, Hammersmith and Fulham",Hammersmith and Fulham
3,City,City of London
4,Westminster,Westminster
...,...,...
374,Redbridge,Redbridge
375,"Redbridge, Waltham Forest",Redbridge
376,Barnet,Barnet
377,Greenwich,Greenwich


In [ ]:
df_geo.head(n=20)

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude,matches,dummy,Most_Similar
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.079100,"Bexley, Greenwich",True,Bexley
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W3,"Ealing, Hammersmith & West London College, St ...","(51.49153695, -0.21497089652160112, 0.0)",51.491537,-0.214971,"Hammersmith and Fulham, Ealing",True,Hammersmith and Fulham
2,Acton,"Ealing, Hammersmith and Fulham",LONDON,TQ205805,W4,"Ealing, Hammersmith & West London College, St ...","(51.49153695, -0.21497089652160112, 0.0)",51.491537,-0.214971,"Hammersmith and Fulham, Ealing",True,Hammersmith and Fulham
3,Aldgate,City,LONDON,TQ334813,EC3,"City of London, Greater London, England, Unite...","(51.5156177, -0.0919983, 0.0)",51.515618,-0.091998,City of London,True,City of London
4,Aldwych,Westminster,LONDON,TQ307810,WC2,"Westminster, Millbank, City of Westminster, Lo...","(51.5004439, -0.1265398, 0.0)",51.500444,-0.126540,Westminster,True,Westminster
5,Anerley,Bromley,LONDON,TQ345695,SE20,"Bromley, London, Greater London, England, BR1 ...","(51.4028046, 0.0148142, 0.0)",51.402805,0.014814,Bromley,True,Bromley
6,Angel,Islington,LONDON,TQ345665,EC1,"Islington, London, Greater London, England, N1...","(51.5384287, -0.0999051, 0.0)",51.538429,-0.099905,Islington,True,Islington
7,Angel,Islington,LONDON,TQ345665,N1,"Islington, London, Greater London, England, N1...","(51.5384287, -0.0999051, 0.0)",51.538429,-0.099905,Islington,True,Islington
8,Archway,Islington,LONDON,TQ285875,N19,"Islington, London, Greater London, England, N1...","(51.5384287, -0.0999051, 0.0)",51.538429,-0.099905,Islington,True,Islington
9,Arkley,Barnet,"BARNET, LONDON",TQ225955,EN5,"Chipping Barnet, London, Greater London, Engla...","(51.65309, -0.2002261, 0.0)",51.653090,-0.200226,Barnet,True,Barnet


Proper Merging on the Basis of the Same column values of Most Similar and Area

In [ ]:
merge = pd.merge(df_geo, df2, left_on=['Most_Similar'],right_on=['Area'],how='outer').fillna(0)

In [ ]:
merge.head()

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude,matches,dummy_x,Most_Similar,Code,Area,InnerOuter,PopulationEstimate_2014,HouseholdEstimate_2014,InlandArea_Hectares,PopulationDensity2013,AverageAge2013,ProportionPopulation0to15,ProportionWorkingage2013,Proportion65over2013,ResidentPopulation_abroad2013,LargestMigrantPopulation2011,PercentLargestMigrant2011,OverseasNationalsEntering2013,Employmentrate2013,MaleEmployment2013,FemaleEmployment2013,WorkingAgeBenefits,WorkingAgeDisability,ProportionDegreeHolders2013,GrossAnnualPay2014,GrossAnnualPay_Male2014,GrossAnnualPay_Female2014,ActiveBusinesses2013,MedianHousePrice2013,NewHomes,Greenspace,CarbonEmissions,NumberCars,CarsPerHouseold,AveragePublicTransportAccessibility 2012,MultipleDeprivationScore,MaleLifeExpectancy,FemaleLifeExpectancy,TeenageConception,LifeSatisfaction,Worthwhileness,Happiness,PoliticalControl,dummy_y
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.079100,"Bexley, Greenwich",True,Bexley,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
1,Bexleyheath (also Bexley New Town),Bexley,"BEXLEYHEATH, LONDON",TQ485755,DA6,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,Bexley,True,Bexley,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
2,Bexleyheath (also Bexley New Town),Bexley,"BEXLEYHEATH, LONDON",TQ485755,DA7,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,Bexley,True,Bexley,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
3,Bexleyheath (also Bexley New Town),Bexley,"BEXLEYHEATH, LONDON",TQ485755,SE2,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,Bexley,True,Bexley,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
4,Crossness,Bexley,LONDON,TQ480800,SE2,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,Bexley,True,Bexley,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True


In [ ]:
clean = merge.drop(['matches','Most_Similar','dummy_x'], axis=1)

In [ ]:
clean.head()

,Location,Borough,Town,OS,Postalcode,location,point,latitude,longitude,Code,Area,InnerOuter,PopulationEstimate_2014,HouseholdEstimate_2014,InlandArea_Hectares,PopulationDensity2013,AverageAge2013,ProportionPopulation0to15,ProportionWorkingage2013,Proportion65over2013,ResidentPopulation_abroad2013,LargestMigrantPopulation2011,PercentLargestMigrant2011,OverseasNationalsEntering2013,Employmentrate2013,MaleEmployment2013,FemaleEmployment2013,WorkingAgeBenefits,WorkingAgeDisability,ProportionDegreeHolders2013,GrossAnnualPay2014,GrossAnnualPay_Male2014,GrossAnnualPay_Female2014,ActiveBusinesses2013,MedianHousePrice2013,NewHomes,Greenspace,CarbonEmissions,NumberCars,CarsPerHouseold,AveragePublicTransportAccessibility 2012,MultipleDeprivationScore,MaleLifeExpectancy,FemaleLifeExpectancy,TeenageConception,LifeSatisfaction,Worthwhileness,Happiness,PoliticalControl,dummy_y
0,Abbey Wood,"Bexley, Greenwich",LONDON,TQ465785,SE2,"Bexley Road, Avery Hill, Royal Borough of Gree...","(51.4510531, 0.0790997, 0.0)",51.451053,0.079100,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
1,Bexleyheath (also Bexley New Town),Bexley,"BEXLEYHEATH, LONDON",TQ485755,DA6,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
2,Bexleyheath (also Bexley New Town),Bexley,"BEXLEYHEATH, LONDON",TQ485755,DA7,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
3,Bexleyheath (also Bexley New Town),Bexley,"BEXLEYHEATH, LONDON",TQ485755,SE2,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
4,Crossness,Bexley,LONDON,TQ480800,SE2,"Bexley, Franklin County, Ohio, 43209, United S...","(39.9692378, -82.936864, 0.0)",39.969238,-82.936864,E09000004,Bexley,Outer London,"2,36,200","93,859",6058.1,39.0,38.9,20.3,63.1,16.6,16.5,Nigeria,2.6,"1,401",72.2,79.5,65.1,10.4,20.2,21.9,"_28,579","_31,374","_23,988","7,830","2,25,000",530,31.7,"1,096.00","1,08,507",1.2,2.5,174,80.3,84.3,25.8,7.26,7.76,7.29,Cons,True
